# ch4. 데이터 전처리 part 1
이전 챕터들에서는 pandas dataframe의 기본 사용법에 대해서 알아보았습니다. 이번 챕터에서는 titanic 데이터 셋을 가지고 생존율 예측 모델을 만든다고 가정하고 데이터 전처리를 해보겠습니다. 특히 데이터 셋에 비어있는 값인 결측치를 해결하는 방법과 카테고리형 데이터를 인코딩 하는 방법에 대해서 알아보겠습니다. 

**사용하는 pandas 기능**
- df.drop
- df.isnull
- df.dropna
- df.fillna
- df.groupby

In [1]:
import pandas as pd

train_df = pd.read_csv("./data/titanic_train.csv", encoding="utf-8")
test_df = pd.read_csv("./data/titanic_test.csv", encoding="utf-8")

In [2]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [3]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


## 전처리를 위해서 하나로 합쳐주기

학습용 데이터와 테스트용 데이터가 PassengerId 891을 기준으로 분리되어 있습니다. 이를 합쳐서 전처리 해준 뒤, 다시 쪼개주겠습니다. test_df는 우리가 생존 여부를 맞춰야 하는 데이터 셋으로, Survived 컬럼이 Nan이 뜨는 것이 정상입니다.

In [4]:
df = pd.concat([train_df, test_df]).reset_index(drop=True)

In [5]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
1305,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
1306,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
1307,1308,NaN,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


## 사용하지 않는 컬럼 삭제
승객의 정보로 생존율을 예측하는 모델을 만든 다고 했을 때, 이름, 티켓 번호, 객실 이름은 크게 상관이 없어 보입니다. 물론 이 정보들을 활용할 수 있는 방법들도 있겠습니다만, 이 가이드에서는 날려버리겠습니다.

In [6]:
df = df.drop(["Name", "Ticket", "Cabin"], axis=1)

In [7]:
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0.0,3,male,22.0,1,0,7.2500,S
1,2,1.0,1,female,38.0,1,0,71.2833,C
2,3,1.0,3,female,26.0,0,0,7.9250,S
3,4,1.0,1,female,35.0,1,0,53.1000,S
4,5,0.0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,male,NaN,0,0,8.0500,S
1305,1306,NaN,1,female,39.0,0,0,108.9000,C
1306,1307,NaN,3,male,38.5,0,0,7.2500,S
1307,1308,NaN,3,male,NaN,0,0,8.0500,S


## 파생 변수 생성하기
주어진 피쳐들을 가공하여 새로운 피쳐를 만들 수 있습니다. 이렇게 만든 피쳐를 파생 변수라고 부릅니다. 타이타닉 데이터 셋에 대해서 파생 변수를 만들어보겠습니다.  

파생 변수를 만들 때에는 먼저 가설을 세우고 그에 맞는 변수를 생성하면 됩니다. 여기서는 "혼자 탑승한 승객은 가족과 함께 탑승한 승객보다 생존률이 높을 것이다"라는 가설을 세우고, 이를 잘 설명할 수 있는 파생 변수를 만들어 보겠습니다. SibSp, Parch는 각각 동승한 배우자나 형제자매의 수, 부모 또는 아이의 수입니다. 즉, SibSp와 Parch 합이 0인 사람은 혼자 탑승한 사람, 그렇지 않으면 동승자가 있는 사람이 됩니다.

In [8]:
df["FamilySize"] = df["SibSp"] + df["Parch"]

In [9]:
df["IsAlone"] = 1

In [10]:
df.loc[df["FamilySize"] >= 1, "IsAlone"] = 0

In [11]:
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,1,0.0,3,male,22.0,1,0,7.2500,S,1,0
1,2,1.0,1,female,38.0,1,0,71.2833,C,1,0
2,3,1.0,3,female,26.0,0,0,7.9250,S,0,1
3,4,1.0,1,female,35.0,1,0,53.1000,S,1,0
4,5,0.0,3,male,35.0,0,0,8.0500,S,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,male,NaN,0,0,8.0500,S,0,1
1305,1306,NaN,1,female,39.0,0,0,108.9000,C,0,1
1306,1307,NaN,3,male,38.5,0,0,7.2500,S,0,1
1307,1308,NaN,3,male,NaN,0,0,8.0500,S,0,1


데이터 셋에 원래 주어졌던 SibSp, Parch 컬럼을 이용해서 FamilySize와 IsAlone이라는 파생 변수를 만들었습니다.

## 결측값 처리하기
데이터 베이스를 공부했을 때, titanic 데이터 셋을 테이블에 집어 넣을 때 Age 값이 빠져있어서 애먹었던 기억이 있으실 겁니다. 바로 이렇게 데이터 셋에서 일부 누락된 값들을 결측치라고 부릅니다. pandas를 이용하면 이런 결측치들을 쉽게 파악할 수 있고, 결측값을 데이터 셋에서 제외하거나 적절한 값으로 채워넣을 수도 있습니다.

### 결측치 확인
데이터 프레임이 주어졌을 때, 가장 먼저 확인해봐야 할 것은 각 컬럼별로 결측값이 얼마나 포함되었는지 입니다. df.isnull()을 사용해서 쉽게 구할 수 있습니다.

In [12]:
df.isnull().sum()

PassengerId      0
Survived       418
Pclass           0
Sex              0
Age            263
SibSp            0
Parch            0
Fare             1
Embarked         2
FamilySize       0
IsAlone          0
dtype: int64

실제로 확인해본 결과, Age, Embarked 컬럼에 결측값을 발견했습니다. 본격적으로 예측 모델을 만들기 전에, 이 결측값들을 미리 해결해야 합니다.

### 결측값 처리 1. 날려버리기
결측값을 처리하는 가장 손쉬운 방법은 결측값이 하나라도 포함된 행이나 열을 날려버리는 것입니다. Age나 Embarked가 비어있는 행만 따로 날려보도록 하겠습니다. 이 때, df.drop, df.dropna 함수를 사용하면 됩니다.

In [13]:
df.dropna(subset=["Age", "Embarked", "Fare"])

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,1,0.0,3,male,22.0,1,0,7.2500,S,1,0
1,2,1.0,1,female,38.0,1,0,71.2833,C,1,0
2,3,1.0,3,female,26.0,0,0,7.9250,S,0,1
3,4,1.0,1,female,35.0,1,0,53.1000,S,1,0
4,5,0.0,3,male,35.0,0,0,8.0500,S,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1300,1301,NaN,3,female,3.0,1,1,13.7750,S,2,0
1302,1303,NaN,1,female,37.0,1,0,90.0000,Q,1,0
1303,1304,NaN,3,female,28.0,0,0,7.7750,S,0,1
1305,1306,NaN,1,female,39.0,0,0,108.9000,C,0,1


결측값이 있는 행을 모조리 날려버려서 결측치 없는 데이터 프레임을 얻을 수 있었습니다.

### 결측값 처리 2. 임의의 값을 채워넣기
가뜩이나 없는 데이터를 Age 값 하나 빠졌다고 100개가 넘는 행을 날려버리는 것은 무척이나 아깝습니다. 그럴듯한 값으로 채워넣으면 충분히 살릴 수 있지 않을까요?

series.fillna를 사용하면 nan 값을 특정 값으로 채워넣을 수 있습니다. 한번 비어있는 Age 값을 100으로 채워보겠습니다.

In [14]:
df["Age"].fillna(100)

0        22.0
1        38.0
2        26.0
3        35.0
4        35.0
        ...  
1304    100.0
1305     39.0
1306     38.5
1307    100.0
1308    100.0
Name: Age, Length: 1309, dtype: float64

비어있던 Age 값이 100으로 채워졌습니다. 비어있는 값을 채워넣더라도 그럴싸한 값으로 채워넣으면 어떨까요? 예를들어 100으로 채워넣기보단 전체 승객의 평균 연령으로 채워넣는 것이 더 좋아보입니다. dataframe의 groupby를 사용하면 쉽게 그룹별 평균을 계산할 수 있습니다.

In [15]:
df.groupby(["Sex"])["Age"].mean()

Sex
female    28.687088
male      30.585228
Name: Age, dtype: float64

남성 평균 연령은 30.585228세, 여성 평균 연령은 28.687088세로 집계되었습니다. 이를 좌석 등급별로 한번 더 나눠보면 더 정확하지 않을까요? 한번 해보겠습니다.

In [16]:
df.groupby(["Sex", "Pclass"])["Age"].mean()

Sex     Pclass
female  1         37.037594
        2         27.499223
        3         22.185329
male    1         41.029272
        2         30.815380
        3         25.962264
Name: Age, dtype: float64

transform 함수를 이용하면 각 행별로 해당하는 group의 평균 나이를 알 수 있습니다.

In [17]:
 df["Age"] = df["Age"].fillna(df.groupby(["Sex", "Pclass"])["Age"].transform("mean"))

Age 결측치를 모두 채워넣어 봤습니다. Embarked 결측치는 가장 많이 탑승한 승선지인 S,   
Fare 결측치는 전체 운임 평균으로 채워넣겠습니다. 

In [18]:
df["Embarked"] = df["Embarked"].fillna("S")
df["Fare"] = df["Fare"].fillna(df["Fare"].mean())

In [19]:
df.isnull().sum()

PassengerId      0
Survived       418
Pclass           0
Sex              0
Age              0
SibSp            0
Parch            0
Fare             0
Embarked         0
FamilySize       0
IsAlone          0
dtype: int64

지금까지 결측치를 다루는 대표적인 2가지 방법을 알아보았습니다. 이 외에도 결측치를 그냥 아예 하나의 값으로 두는 방법, 머신러닝 모델로 최적의 결측치를 예측하는 방법 등이 있습니다. 정답이 정해져 있는 것이 아니고, 풀어야 하는 문제와 데이터 셋의 특성에 따라서 적절한 방식을 선택하면 됩니다.

## 카테고리형 데이터 인코딩
사용 안하는 컬럼을 제거하고, 결측값을 메꾼 데이터프레임을 얻었습니다.  

In [20]:
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone
0,1,0.0,3,male,22.000000,1,0,7.2500,S,1,0
1,2,1.0,1,female,38.000000,1,0,71.2833,C,1,0
2,3,1.0,3,female,26.000000,0,0,7.9250,S,0,1
3,4,1.0,1,female,35.000000,1,0,53.1000,S,1,0
4,5,0.0,3,male,35.000000,0,0,8.0500,S,0,1
...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,male,25.962264,0,0,8.0500,S,0,1
1305,1306,NaN,1,female,39.000000,0,0,108.9000,C,0,1
1306,1307,NaN,3,male,38.500000,0,0,7.2500,S,0,1
1307,1308,NaN,3,male,25.962264,0,0,8.0500,S,0,1


### Label Encoding
먼저 성별을 보면 male, female과 같이 문자열로 표시되어 있습니다.  
이는 남성인지 아닌지를 의미하는 is_male이라는 컬럼에 0과 1로 표현할 수 있습니다.  
scikit-learn의 label encoder를 사용하면 간편하게 구현할 수 있습니다.

In [21]:
!pip install scikit-learn


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [22]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

In [23]:
df["is_male"] = label_encoder.fit_transform(df["Sex"])

In [24]:
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,is_male
0,1,0.0,3,male,22.000000,1,0,7.2500,S,1,0,1
1,2,1.0,1,female,38.000000,1,0,71.2833,C,1,0,0
2,3,1.0,3,female,26.000000,0,0,7.9250,S,0,1,0
3,4,1.0,1,female,35.000000,1,0,53.1000,S,1,0,0
4,5,0.0,3,male,35.000000,0,0,8.0500,S,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,male,25.962264,0,0,8.0500,S,0,1,1
1305,1306,NaN,1,female,39.000000,0,0,108.9000,C,0,1,0
1306,1307,NaN,3,male,38.500000,0,0,7.2500,S,0,1,1
1307,1308,NaN,3,male,25.962264,0,0,8.0500,S,0,1,1


### One Hot Encoding
앞서 범주형 데이터 컬럼들을 숫자로 변환했습니다. 

성별 외에도 승선지와 좌석 등급 컬럼이 카테고리형 컬럼입니다. 그런데 카테고리 데이터에도 종류가 있습니다.  
먼저 좌석 등급처럼 값들간의 순서가 있는 서열형 데이터입니다.  
그리고 승선지 처럼 값들간의 순서가 없는 단순 카테고리 데이터가 있습니다.

서열형 데이터의 경우에는 그대로 놔둬도 되지만, 그렇지 않은 데이터는 one-hot 인코딩을 적용해주는 것이 좋습니다.
one-hot encoding은 각 범주별로 컬럼을 쪼개고, 해당 범주에 속하는지 여부만 0과 1로 표현하도록 변환합니다.
pandas의 dummies 함수를 이용하여 쉽게 구현할 수 있습니다.

In [25]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,is_male
0,1,0.0,3,male,22.0,1,0,7.2500,S,1,0,1
1,2,1.0,1,female,38.0,1,0,71.2833,C,1,0,0
2,3,1.0,3,female,26.0,0,0,7.9250,S,0,1,0
3,4,1.0,1,female,35.0,1,0,53.1000,S,1,0,0
4,5,0.0,3,male,35.0,0,0,8.0500,S,0,1,1


In [26]:
embarked_dummies = pd.get_dummies(df["Embarked"], prefix="Embarked")

In [29]:
embarked_dummies

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1
...,...,...,...
1304,0,0,1
1305,1,0,0
1306,0,0,1
1307,0,0,1


one-hot encoding을 적용한 컬럼들을 기존 데이터 프레임에 합쳐줍니다.  
이 때, df.concat을 사용합니다. 행을 붙이는 것이 아닌, 컬럼을 붙이는 것이므로 axis=1로 설정해줍니다.

In [36]:
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,is_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0.0,3,male,22.000000,1,0,7.2500,S,1,0,1,0,0,1
1,2,1.0,1,female,38.000000,1,0,71.2833,C,1,0,0,1,0,0
2,3,1.0,3,female,26.000000,0,0,7.9250,S,0,1,0,0,0,1
3,4,1.0,1,female,35.000000,1,0,53.1000,S,1,0,0,0,0,1
4,5,0.0,3,male,35.000000,0,0,8.0500,S,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,male,25.962264,0,0,8.0500,S,0,1,1,0,0,1
1305,1306,NaN,1,female,39.000000,0,0,108.9000,C,0,1,0,1,0,0
1306,1307,NaN,3,male,38.500000,0,0,7.2500,S,0,1,1,0,0,1
1307,1308,NaN,3,male,25.962264,0,0,8.0500,S,0,1,1,0,0,1


### 전처리를 마친 컬럼 제거
one-hot encoding을 적용한 Pclass, Embarked 컬럼은 제거

In [37]:
df = df.drop(["Sex", "Embarked"], axis=1)

In [38]:
df

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,FamilySize,IsAlone,is_male,Embarked_C,Embarked_Q,Embarked_S
0,1,0.0,3,22.000000,1,0,7.2500,1,0,1,0,0,1
1,2,1.0,1,38.000000,1,0,71.2833,1,0,0,1,0,0
2,3,1.0,3,26.000000,0,0,7.9250,0,1,0,0,0,1
3,4,1.0,1,35.000000,1,0,53.1000,1,0,0,0,0,1
4,5,0.0,3,35.000000,0,0,8.0500,0,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,NaN,3,25.962264,0,0,8.0500,0,1,1,0,0,1
1305,1306,NaN,1,39.000000,0,0,108.9000,0,1,0,1,0,0
1306,1307,NaN,3,38.500000,0,0,7.2500,0,1,1,0,0,1
1307,1308,NaN,3,25.962264,0,0,8.0500,0,1,1,0,0,1


## 결과 저장

전처리 과정이 아직 남아있습니다만, 이번 챕터가 너무 길어져서 끊고 가겠습니다. 데이터 프레임의 to_csv() 함수를 이용하면 쉽게 데이터 프레임을 저장할 수 있습니다.

In [42]:
df.to_csv("./data/tmp.csv", index=False, encoding="utf-8")

## 정리
지금까지 타이타닉 생존자 데이터 셋을 pandas data frame으로 만들어서 여러 전처리를 해보았습니다. 사용하지 않는 컬럼을 삭제하고, 파생 변수를 만들고, 결측치를 해결하고, 변수들을 숫자로 인코딩 하는 등의 작업이 포함되었습니다. 이 모든 과정을 묶어서 피쳐 엔지니어링이라고 부릅니다.

워낙 유명한 예제여서 찾아보시면 다양한 예제들과 기법들이 나올 것입니다. 정해진 정답은 없다는걸 기억하고, 적절해 보이는 기법들을 취사 선택해서 이용하면 됩니다.